In [ ]:
!mkdir dataset

In [5]:
from google.colab import files
src = list(files.upload().values())[0]
open('\dataset','wb').write(src)

ModuleNotFoundError: No module named 'google'

In [ ]:
!mv sample_mobile_data_1000.csv dataset/

In [ ]:
!ls dataset

In [ ]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [2]:
from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google'

In [3]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

ModuleNotFoundError: No module named 'oauth2client'

In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
 
print 'Files in Drive:'
!ls drive/
 
# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Files in Drive:')? (<ipython-input-4-fd16e6c8b3f6>, line 4)

In [ ]:
!ls
